In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
from __future__ import print_function

# # Imports
import sys, time
import pprint
import json
from pandda_2 import (config,
                      pandda_phil,
                      options,
                      load_dataset,
                      transform_dataset,
                      get_dataset,
                      get_reference,
                      get_grid,
                      define_tree,
                      make_tree,
                      copy_dataset_files,
                      output,
                      create_shells,
                      map_loader,
                      get_reference_map,
                      statistical_model,
                      fit_model,
                      evaluate_model,
                      cluster_outliers,
                      filter_clusters,
                      event_analyser,
                      make_event_map,
                      make_z_map,
                      make_mean_map,
                      make_event_table,
                      process_shell,
                      process_shells,
                      processor,
                      create_sites_table,
                      output_sites_table,
                      create_event_table,
                      output_event_table,
                      standard_pandda,
                      autobuild,
                      )


class TaskWrapper:
    def __init__(self, func, *args, **kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs

    def __call__(self):
        return self.func(*self.args, **self.kwargs)



In [55]:
import os, subprocess, pathlib

import numpy as np
from biopandas.pdb import PandasPdb

from scipy.spatial.transform import Rotation as R
from scipy.optimize import differential_evolution

import iotbx


class QFitParameters:
    def __init__(self,
                 density_cutoff,
                 ):
        self.density_cutoff = 0


class InitialModel:
    def __init__(self,
                 ):
        pass
    
    def __call__(self, 
                 ligand_model_path,
                 output_dir_path,
                 event_map_path,
                 event,
                 index,
                ):
        
#         placed_ligand_model_path = get_placed_ligand_model_path(ligand_model_path,
#                                                                index,
#                                                                )

        place_ligand(ligand_model_path,
                     event,
                     output_dir_path,
                     event_map_path,
                     )
        
        return output_dir_path
    
class StripProtein:
    def __init__(self,
                 ):
        pass
    
    def __call__(self, 
                protein_model_path,
                 output_path,
                ):
        
        protein_stripped_path = get_protein_stripped_path(protein_model_path)
        
        strip_protein(protein_model_path,
                     output_path,
                     )
        
        return output_dir_path
    
class AutobuildQFit:
    def __init__(self,
                 ):
        pass

    def __call__(self,
                 protein_model_path,
                 ligand_model_path,
                 event_map_path,
                 output_dir_path,
                 resolution,
                 event,
                 index,
                 qfit_parameters=QFitParameters(density_cutoff=0),
                 ):


        command_string = "/dls/science/groups/i04-1/conor_dev/ccp4/base/bin/qfit_ligand {event_map} {resolution} {ligand_model_path} -r {protein_model_path} -d {output_dir_path}"
        run_string = command_string.format(event_map=event_map_path,
                              resolution=resolution,
                              ligand_model_path=placed_ligand_model_path,
                              protein_model_path=protein_stripped_path,
                              output_dir_path=output_dir_path,
                              )

        print("Run string: {}".format(run_string))
        
        p = subprocess.Popen(run_string,
                             shell=True,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE,
                             )

        stdout, stderr = p.communicate()
        
        
def get_protein_stripped_path(protein_model_path):
    stripped_protein_model_path = pathlib.Path(protein_model_path).parent / "stripped_protein.pdb"
    return stripped_protein_model_path
    
    
def strip_protein(protein_model_path,
                  protein_stripped_path,
                 ):
    
    protein = PandasPdb().read_pdb(str(protein_model_path))

#     print(protein.df["HETATM"])
    
    protein.df["HETATM"].drop(protein.df["HETATM"].index, inplace=True)
    
#     print(protein.df["HETATM"])

    
    protein.to_pdb(path=str(protein_stripped_path),
              records=None,
              gz=False,
              append_newline=True,
              )

def place_ligand(ligand_model_path,
                 event,
                 output_path,
                 event_map_path,
                 ):
    
    xmap = get_xmap(event_map_path)
#     print(dir(xmap))#
#     print(xmap.grid_unit_cell())
#     print(xmap.unit_cell_parameters)
#     print(xmap.unit_cell())
#     print(dir(xmap.unit_cell()))
    unit_cell = xmap.unit_cell()# .fractionalization_matrix()
#     print(unit_cell)
#     print(dir(unit_cell))
    
    ligand = PandasPdb().read_pdb(str(ligand_model_path))

    ligand_coords = ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]]
    ligand_coords_np = ligand_coords.to_numpy()
#     print(ligand_coords)

    ligand_com = ligand_coords_np.mean(axis=0)


#     print("Event is: {}".format(event))
    event_com = np.array([event["x"], event["y"], event["z"]])

#     print(event_com)
#     print(ligand_com)
    translation = event_com - ligand_com
    
#     print(ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]])
    ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]] += translation
#     print(ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]])
    
    new_ligand, score = build_ligand(xmap,
                              ligand,
                              unit_cell,
                             )
    
    print("\tOutputting ligand to: {}".format(output_path))
    new_ligand.to_pdb(path=str(output_path),
                  records=None,
                  gz=False,
                  append_newline=True,
                  )
    
    return score


def get_placed_ligand_model_path(ligand_model_path,
                                index,
                                ):
    placed_ligand_model_path = pathlib.Path(ligand_model_path).parent.parent / "autobuilt/placed_ligand_{}_{}.pdb".format(index[0],
                                                                                                                         index[1],
                                                                                                                         )
    return placed_ligand_model_path


def build_ligand(xmap, 
                 ligand, 
                 unit_cell,
                 optimise=differential_evolution, 
                 bounds=[(-3, 3), (-3, 3), (-3, 3), (0, 2*np.pi), (0, 2*np.pi), (0, 2*np.pi),],
                ):
        
    ligand_coords = ligand.df["HETATM"][ligand.df["HETATM"]["element_symbol"] != "H"][["x_coord", "y_coord", "z_coord"]].to_numpy()
    
#     model = LigandModel(ligand_coords)

#     print("FRACTIONALISED: {}".format(xmap.unit_cell().fractionalize([0,1,0])))

    scoring_function = ScoreModel(ligand_coords, xmap, unit_cell)
    
#     print(scoring_function([0,0,0,0,0,0]))
    
    optimised_model = optimise(scoring_function,
                               disp=True,
                              bounds=bounds,
                               maxiter=80,
                              )
    
#     print(optimised_model)
    params = optimised_model.x
    score = optimised_model.fun
    translation = (params[0], params[1], params[2])
    rotation = (params[3], params[4], params[5])
    
    
#     print(ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]])
    
#     print(ligand_coords)
    new_coords = rotate_translate(ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]].to_numpy(),
                     rotation=rotation,
                     translation=translation,
                    )
#     print(new_coords)
    
#     print(ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]])
    ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]] = new_coords                                                                                                 
#     print(ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]])

    
    return ligand, score
    
    


class LigandModel:
    def __init__(self, coords):
        
        # coords = ligand.df["HETATM"][["x_coord", "y_coord", "z_coord"]].to_numpy()
        
        self.coords = coords
        self.location = [0,0,0]
        self.rotation = [0,0,0]
        
def rotate_translate(coords, rotation=[0,0,0], translation=[0,0,0]):
    
    coords_com = np.mean(coords, axis=0)
                 
    rotation = R.from_euler("xyz", rotation)
        
    new_coords = rotation.apply(coords - coords_com) + coords_com + translation
    
    return new_coords


class ScoreModel:
    def __init__(self, coords, xmap, unit_cell):
        self.coords = coords
        self.xmap_np = xmap
        self.unit_cell = unit_cell
#         self.world_to_grid = 
#         self.interpolator = RegularGridInterpolator((x, y, z), 
#                                                    data)
        
    def __call__(self, params):
        
        return self.score_model(self.coords, self.xmap_np, params)
        
    def score_model(self, coords, xmap, params):
    
        translation = (params[0], params[1], params[2])
        rotation = (params[3], params[4], params[5])
        
        coords_for_translation = coords.copy()
        
        transformed_coords = rotate_translate(coords_for_translation,
                                              rotation=rotation,
                                             translation=translation,
                                             )
        
#         print("Fractinolizing coords!")
#         print(transformed_coords)
#         print(self.unit_cell.fractionalize([0,1,0]))
        frac_values = [self.unit_cell.fractionalize(coord) for coord in transformed_coords]
#         frac_values = [np.matmul(self.unit_cell, coord) for coord in transformed_coords]
        
#         print(frac_values)
#         print(dir(xmap.map_data()))
        sample_points = map(xmap.map_data().eight_point_interpolation,
                            [coord for coord in frac_values],
                           )
        
        sample_points_array = np.array(sample_points)
        
#         sample_points_array[sample_points_array < 0.6] = 0
#         sample_points_array[sample_points_array > 0.6] = 1
        
        sample_points_array = -sample_points_array
#         sample_points[sample_points < 2] = 0
        
#         print(sample_points)
        
        return sum(sample_points_array)
                 
    
#     grid_coords = world_to_grid(coords,
#                                self.world_to_grid,
#                                )
                 
#     mask = mask_coords(coords)
    
#     masked_coords = xmap[coords]
    
#     score = np.sum(masked_coords > cutoff)
                 
#     interpolated_values = self.interpolator(coords)
    
        
def get_xmap(xmap_path):
    
    xmap = iotbx.ccp4_map.map_reader(str(xmap_path))
                 
    return xmap


# def get_world_to_grid(grid_params,
#                      unit_cell_params,
#                       origin,
#                      ):
    
                 
                 
# def world_to_grid(coords, 
#                   world_to_grid_translation,
#                   world_to_grid_rotation,
#                  ):
#     return np.matmul(coords - np.mean(coords, axis=0), world_to_grid_rotation) + world_to_grid_translation
    
    

In [56]:
class RealSpaceRefinePhenix:
    def __init__(self):
        pass
    def __call__(self,
                event_map_path,
                resolution,
                 placed_ligand_model_path,
                 output_dir_path,
                ):
        command_string = "cd {output_dir}; module load phenix; phenix.ready_set {ligand_model} output_file_name=ready; phenix.real_space_refine {event_map} ready.pdb ready.cif resolution={resolution}"
        run_string = command_string.format(event_map=event_map_path,
                              resolution=resolution,
                              ligand_model=placed_ligand_model_path,
                              output_dir=output_dir_path,
                              )

        print("Run string: {}".format(run_string))
        
        p = subprocess.Popen(run_string,
                             shell=True,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE,
                             )

        stdout, stderr = p.communicate()

In [57]:
args = ["data_dirs=/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/*",
        "pdb_style=*.dimple.pdb", 
        "mtz_style=*.dimple.mtz" ,
        "cpus=21",
        "out_dir=/dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B", 
        "diffraction_data.structure_factors=2FOFCWT,PH2FOFCWT"]

In [58]:

# Get start time
pandda_start_time = time.time()

# Parse Config files and command line arguments
working_phil = config.extract_params_default(master_phil=pandda_phil.pandda_phil,
                                             args=args,
                                             blank_arg_prepend=None,
                                             home_scope=None
                                             ).extract()

# Maps options to code abstraction: Phil -> Config
config = config.Config(working_phil)

# Options: maps a config to options
options = options.Options(config)

In [59]:


# Get dataset loader
load_dataset = load_dataset.LoadDataset(dataloader=options.dataloader,
                                        sample_loader=options.sample_loader,
                                        )

# Get reference loader
get_reference = options.get_reference

# Get dataset transformer
transform_dataset = transform_dataset.TransformDataset(transform_data_check=options.transform_data_check,
                                                       transform_scale_diffraction=options.transform_scale_diffraction,
                                                       transform_filter_structure=options.transform_filter_structure,
                                                       transform_filter_wilson=options.transform_filter_wilson,
                                                       transform_align=options.transform_align,
                                                       )

# Get grid loader
get_grid = options.grid_loader

# Get partitioner
partitioner = options.partitioner

# Get output handler
define_tree = define_tree.DefineTree(output_dir=config.output.out_dir)
make_tree = make_tree.MakeTree(overwrite=config.output.overwrite)
copy_dataset_files = copy_dataset_files.DatasetFileCopier()

output = output.Output(define_tree=define_tree,
                       make_tree=make_tree,
                       copy_dataset_files=copy_dataset_files,
                       )

# Get resolution shell scheme
create_shells = create_shells.CreateShells(min_train_datasets=60,
                                           max_test_datasets=60,
                                           cutoff=0.1,
                                           )

# Get Resolution shell processor
diffraction_data_truncator = options.diffraction_data_truncator
reference_map_getter = options.reference_map_getter
get_reference_map = get_reference_map.GetReferenceMap()

map_loader = options.map_loader
statistical_model = options.statistical_model
fit_model = fit_model.FitModel()
evaluate_model = evaluate_model.EvaluateModel()
cluster_outliers = options.clusterer
filter_clusters = options.event_finder
event_analyser = options.event_analyser

make_event_map = make_event_map.MakeEventMap()
make_z_map = make_z_map.MakeZMap()
make_mean_map = make_mean_map.MakeMeanMap()
make_event_table = make_event_table.MakeEventTable()

process_in_shell = processor.ProcessorDictJoblib()

process_shell = process_shell.ProcessShell(diffraction_data_truncator=diffraction_data_truncator,
                                           reference_map_getter=reference_map_getter,
                                           get_reference_map=get_reference_map,
                                           map_loader=map_loader,
                                           statistical_model=statistical_model,
                                           fit_model=fit_model,
                                           evaluate_model=evaluate_model,
                                           cluster_outliers=cluster_outliers,
                                           filter_clusters=filter_clusters,
                                           event_analyser=event_analyser,
                                           make_event_map=make_event_map,
                                           make_z_map=make_z_map,
                                           make_mean_map=make_mean_map,
                                           make_event_table=make_event_table,
                                           process=process_in_shell,
                                           )

processer_shells = processor.ProcessorLuigi(jobs=10,
                                     parallel_env="smp",
                                     n_cpu=12,
                                     run_locally=False,
                                     )

processer_building = processor.ProcessorDictEasyMP()

# Get site table creator
create_sites_table = create_sites_table.CreateSitesTable()

# Get event table outputter
output_sites_table = output_sites_table.OutputSitesTable()

# Get event table processor
create_event_table = create_event_table.CreateEventTable()

# Get event table outputter
output_event_table = output_event_table.OutputEventTable()

# Autobuilders
autobuilder = AutobuildQFit()



In [8]:
# Define program
print("Loading dataset")
dataset = load_dataset()

Loading dataset


In [9]:

print("Loading reference")
reference = get_reference(dataset.datasets)

Loading reference


In [10]:
print("Transforming dataset")
dataset = transform_dataset(dataset, reference)

Transforming dataset
	Transform: data check
	Transform: scale diffraction
	Transform: filtering structure
	Transform: filtering Wilson


/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/bamboo/stats/__init__.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return 0.6745*devs/mdev


	Transform: aligning dataset


/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/multi_dataset_crystalography/dataset/transforms.py:350: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_all_z') > self.max_wilson_plot_z_score) or \
/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/multi_dataset_crystalography/dataset/transforms.py:352: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_<4A_z') > self.max_wilson_plot_z_score) or \
/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/multi_dataset_crystalography/dataset/transforms.py:354: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_>4A_z') > self.max_wilson_plot_z_score) or \
/dls/science/groups/i0

[Parallel(n_jobs=21)]: Done 106 tasks      | elapsed:   40.2s
[Parallel(n_jobs=21)]: Done 107 tasks      | elapsed:   40.3s
[Parallel(n_jobs=21)]: Done 108 tasks      | elapsed:   40.3s
[Parallel(n_jobs=21)]: Done 109 tasks      | elapsed:   40.3s
[Parallel(n_jobs=21)]: Done 110 tasks      | elapsed:   40.3s
[Parallel(n_jobs=21)]: Done 111 tasks      | elapsed:   40.3s
[Parallel(n_jobs=21)]: Done 112 tasks      | elapsed:   40.3s
[Parallel(n_jobs=21)]: Done 113 tasks      | elapsed:   40.4s
[Parallel(n_jobs=21)]: Done 114 tasks      | elapsed:   40.4s
[Parallel(n_jobs=21)]: Done 115 tasks      | elapsed:   40.4s
[Parallel(n_jobs=21)]: Done 116 tasks      | elapsed:   40.4s
[Parallel(n_jobs=21)]: Done 117 tasks      | elapsed:   40.5s
[Parallel(n_jobs=21)]: Done 118 tasks      | elapsed:   40.5s
[Parallel(n_jobs=21)]: Done 119 tasks      | elapsed:   40.5s
[Parallel(n_jobs=21)]: Done 120 tasks      | elapsed:   40.5s
[Parallel(n_jobs=21)]: Done 121 tasks      | elapsed:   40.5s
[Paralle

In [11]:


print("Partitioning")
dataset.partitions = partitioner(dataset.datasets)

Partitioning


In [12]:
print("Getting grid")
grid = get_grid(reference)

Getting grid
Creating referene grid
Masking reference grid
----------------------------------->>>
Atomic Mask Summary:
Total Mask Size (1D): 287838
Outer Mask Size (1D): 385463
Inner Mask Size (1D): 97625
Masked Grid Min/Max: ((7, 87, 44), (128, 47, 19))
----------------------------------->>>
Atomic Mask Summary:
Total Mask Size (1D): 508968
Outer Mask Size (1D): 1003011
Inner Mask Size (1D): 494043
Masked Grid Min/Max: ((0, 0, 7), (135, 132, 90))
shifting reference origin


[Parallel(n_jobs=21)]: Using backend LokyBackend with 21 concurrent workers.
[Parallel(n_jobs=21)]: Done   4 out of  21 | elapsed:   13.9s remaining:   59.2s
[Parallel(n_jobs=21)]: Done   7 out of  21 | elapsed:   15.8s remaining:   31.6s
[Parallel(n_jobs=21)]: Done  10 out of  21 | elapsed:   18.6s remaining:   20.4s
[Parallel(n_jobs=21)]: Done  13 out of  21 | elapsed:   20.6s remaining:   12.7s
[Parallel(n_jobs=21)]: Done  16 out of  21 | elapsed:   22.9s remaining:    7.1s
[Parallel(n_jobs=21)]: Done  19 out of  21 | elapsed:   23.2s remaining:    2.4s
[Parallel(n_jobs=21)]: Done  21 out of  21 | elapsed:   24.4s finished


STARTING MULTIPROCESSING


In [13]:
print("Partitioning shells")
shells = {shell_num: shell_dataset
          for shell_num, shell_dataset
          in create_shells(dataset)
          }

for shell_num, shell_dataset in shells.items():
    print(shell_dataset.get_partition("test"))
    print(shell_dataset.get_partition("train"))


Partitioning shells
Got all train datasets
Sorted datasets
('BAZ2BA-x645', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cabb90>)
('BAZ2BA-x613', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28d19950>)
('BAZ2BA-x655', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cb9290>)
('BAZ2BA-x667', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cc24d0>)
('BAZ2BA-x654', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cb9050>)
('BAZ2BA-x657', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cb94d0>)
('BAZ2BA-x653', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cb1dd0>)
('BAZ2BA-x510', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28d32b90>)
('BAZ2BA-x640', <multi_dataset_crystalography.dataset.dataset.PanddaDataset object at 0x7f8a28cab050>)
('BAZ2BA-x677'

In [14]:
print("Output")
tree = output(dataset,
              shells,
              )

Output
	Copying pdb for dataset BAZ2BA-x558
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x558/BAZ2BA-x558-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x558
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x558/BAZ2BA-x558-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x558
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x558
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x558/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x558
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x558/autobuilt/ligand.cif
	Converting cifs to li

LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x655/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x655
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x655/autobuilt/ligand.cif
	Converting cifs to ligands for dataset BAZ2BA-x655
	Copying pdb for dataset BAZ2BA-x595
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x595/BAZ2BA-x595-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x595
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x595/BAZ2BA-x595-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x595
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/

LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x644/BAZ2BA-x644-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x644
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x644/BAZ2BA-x644-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x644
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x644
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x644/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x644
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x644/autobuilt/ligand.cif
	Converting cifs to ligands for dataset BAZ2BA-x644
	Copying pdb f

LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x510/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x510
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x510/autobuilt/ligand.cif
	Converting cifs to ligands for dataset BAZ2BA-x510
	Copying pdb for dataset BAZ2BA-x511
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x511/BAZ2BA-x511-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x511
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x511/BAZ2BA-x511-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x511
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/

LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x447/autobuilt/ligand.cif
	Converting cifs to ligands for dataset BAZ2BA-x447
	Copying pdb for dataset BAZ2BA-x448
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x448/BAZ2BA-x448-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x448
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x448/BAZ2BA-x448-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x448
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x448
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x448/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/

LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x529/autobuilt/ligand.cif
	Converting cifs to ligands for dataset BAZ2BA-x529
	Copying pdb for dataset BAZ2BA-x528
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x528/BAZ2BA-x528-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x528
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x528/BAZ2BA-x528-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x528
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x528
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x528/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/

LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x530/autobuilt/ligand.cif
	Converting cifs to ligands for dataset BAZ2BA-x530
	Copying pdb for dataset BAZ2BA-x531
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x531/BAZ2BA-x531-pandda-input.pdb
	Copying mtz for dataset BAZ2BA-x531
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x531/BAZ2BA-x531-pandda-input.mtz
	Copying ligand for dataset BAZ2BA-x531
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/data/BAZ2BA-x531
*.cif
LINK ALREADY EXISTS: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x531/ligand_files/ligand.cif
/dls/science/groups/i04-1/conor_dev/pandda_2_experiments/pandda_2_standard/data/

[Parallel(n_jobs=21)]: Using backend LokyBackend with 21 concurrent workers.
[Parallel(n_jobs=21)]: Done  30 tasks      | elapsed:   29.5s
[Parallel(n_jobs=21)]: Done  71 tasks      | elapsed:  1.0min
[Parallel(n_jobs=21)]: Done 120 tasks      | elapsed:  1.7min
[Parallel(n_jobs=21)]: Done 185 out of 200 | elapsed:  2.4min remaining:   11.8s
[Parallel(n_jobs=21)]: Done 200 out of 200 | elapsed:  3.5min finished


In [15]:
print("Processing shells")
shell_processors = []
for shell_num, shell_dataset in shells.items():
    # if shell_num == 5:
    #     process_shell(shell_dataset = shell_dataset,
    #                   reference = reference,
    #                   grid = grid,
    #                   tree = tree,
    #                   shell_num = shell_num,
    #                   )

    shell_p = TaskWrapper(process_shell,
                          shell_dataset=shell_dataset,
                          reference=reference,
                          grid=grid,
                          tree=tree,
                          shell_num=shell_num,
                          )
    shell_processors.append(shell_p)
event_tables = processer_shells(shell_processors,
                         output_paths=[tree["shells"][shell_num]["event_table"]()
                                       for shell_num
                                       in range(len(shell_processors))
                                       ],
                         result_loader=None,
                         shared_tmp_dir=tree["shells"](),
                         )

/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/luigi/parameter.py:286: UserWarning: Parameter "job_name_format" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/luigi/parameter.py:286: UserWarning: Parameter "output_path" with value "/dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/shells/0/event_table.csv" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/luigi/parameter.py:286: UserWarning: Parameter "func" with value "<__main__.TaskWrapper instance at 0x7f8a219a8bd8>" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
/dls/scien

INFO: Informed scheduler that task   Task____main___TaskWr__dls_science_gro_2dce7b97ce   has status   DONE
DEBUG: Checking if Task(func=<__main__.TaskWrapper instance at 0x7f8a1bfe6710>, output_path=/dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/shells/7/event_table.csv) is complete
INFO: Informed scheduler that task   Task____main___TaskWr__dls_science_gro_d7afb0bd31   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 10 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=536491886, workers=10, host=cs05r-sc-serv-04.diamond.ac.uk, username=zoh22914, pid=215738) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 8 tasks of which:
* 8 complete ones were encountered:
    - 8 Task(...)

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi 

Processing shells


In [16]:
print("Creating event table")
event_table = create_event_table(tree,
                                 len(shells),
                                 )

print(event_table)

Creating event table


/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/pandda_2/create_event_table.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if (len(shell_table) > 0)


[5, 14, 11, 5, 1, 2, 2, 2]
    1-BDC Unnamed: 0  analysed  analysed_resolution  cluster_size  \
0    0.25          0       0.0             1.712564         231.0   
1    0.23          4       0.0             1.712564         226.0   
2    0.23          2       0.0             1.712564          82.0   
3    0.32          3       0.0             1.712564         208.0   
4    0.15          1       0.0             1.712564          87.0   
0    0.26          7       0.0             1.793878         238.0   
1    0.40          8       0.0             1.793878         106.0   
2    0.30          9       0.0             1.793878          96.0   
3    0.24         10       0.0             1.793878          95.0   
4    0.22          1       0.0             1.793878         169.0   
5    0.21          4       0.0             1.793878         223.0   
6    0.27         11       0.0             1.793878         331.0   
7    0.31          2       0.0             1.793878          93.0   
8    0.

In [17]:
print("Creating sites table")
sites_table, events_table_with_sites = create_sites_table(event_table,
                                                          grid,
                                                          reference,
                                                          )

print(events_table_with_sites)

Creating sites table
    1-BDC Unnamed: 0  analysed  analysed_resolution  cluster_size  \
0    0.25          0       0.0             1.712564         231.0   
1    0.23          4       0.0             1.712564         226.0   
2    0.23          2       0.0             1.712564          82.0   
3    0.32          3       0.0             1.712564         208.0   
4    0.15          1       0.0             1.712564          87.0   
0    0.26          7       0.0             1.793878         238.0   
1    0.40          8       0.0             1.793878         106.0   
2    0.30          9       0.0             1.793878          96.0   
3    0.24         10       0.0             1.793878          95.0   
4    0.22          1       0.0             1.793878         169.0   
5    0.21          4       0.0             1.793878         223.0   
6    0.27         11       0.0             1.793878         331.0   
7    0.31          2       0.0             1.793878          93.0   
8    0.29    

In [18]:
from functools import partial

ligand_placers = {}
for index, event in events_table_with_sites.iterrows():
    dtag = index[0]
    analysed_resolution = event["analysed_resolution"]
    bdc = event["1-BDC"]
    event_idx = int(float(index[1]))
    
    ligand_model_paths = list(tree["processed_datasets"][dtag]["ligand_files"]().glob("*.pdb"))
    
    if len(ligand_model_paths) >= 1:
        ligand_model_path = str(ligand_model_paths[0])
    else:
        print("\tNO LIGANDS!")
        continue
        
    event_map_path=tree["processed_datasets"][dtag]["event_map"]([dtag,
                                                                  event_idx,
                                                                  bdc,
                                                                  ],
                                                                 )
        
    output_dir_path=tree["processed_datasets"][dtag]["autobuilt"]["initial_build"]([index[0], index[1]])
    
    ligand_placers[index] = partial(InitialModel(),
                                   ligand_model_path,
                                    output_dir_path,
                                    event_map_path,
                                    event,
                                    index,
                                   )
    


In [19]:
list(ligand_placers.values())[0]()

differential_evolution step 1: f(x)= -4.71421
differential_evolution step 2: f(x)= -5.51357
differential_evolution step 3: f(x)= -5.51357
differential_evolution step 4: f(x)= -5.51357
differential_evolution step 5: f(x)= -5.59539
differential_evolution step 6: f(x)= -6.29262
differential_evolution step 7: f(x)= -6.29262
differential_evolution step 8: f(x)= -6.29262
differential_evolution step 9: f(x)= -6.29262
differential_evolution step 10: f(x)= -6.29262
differential_evolution step 11: f(x)= -6.29262
differential_evolution step 12: f(x)= -6.29262
differential_evolution step 13: f(x)= -6.29262
differential_evolution step 14: f(x)= -6.29262
differential_evolution step 15: f(x)= -6.29262
differential_evolution step 16: f(x)= -6.29262
differential_evolution step 17: f(x)= -6.29262
differential_evolution step 18: f(x)= -6.29262
differential_evolution step 19: f(x)= -6.53321
differential_evolution step 20: f(x)= -6.53321
differential_evolution step 21: f(x)= -6.69076
differential_evolution

/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/biopandas/pdb/pandas_pdb.py:519: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat(dfs)


PosixPath('/dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x447/autobuilt/initial_build_BAZ2BA-x447_1.0.pdb')

In [20]:
placed_ligand_paths = processer_building(ligand_placers)

differential_evolution step 1: f(x)= -6.80544
differential_evolution step 1: f(x)= -12.2691
differential_evolution step 1: f(x)= -4.36519
differential_evolution step 1: f(x)= -8.63129
differential_evolution step 2: f(x)= -6.81803
differential_evolution step 2: f(x)= -12.2691
differential_evolution step 2: f(x)= -4.49099
differential_evolution step 2: f(x)= -8.63129
differential_evolution step 3: f(x)= -6.81803
differential_evolution step 3: f(x)= -12.2691
differential_evolution step 3: f(x)= -5.08415
differential_evolution step 3: f(x)= -8.63129
differential_evolution step 4: f(x)= -7.78305
differential_evolution step 4: f(x)= -12.2691
differential_evolution step 4: f(x)= -8.63129
differential_evolution step 4: f(x)= -5.52985
differential_evolution step 5: f(x)= -7.78305
differential_evolution step 5: f(x)= -12.3059
differential_evolution step 5: f(x)= -8.63129
differential_evolution step 5: f(x)= -5.52985
differential_evolution step 6: f(x)= -7.78305
differential_evolution step 6: f(x

differential_evolution step 34: f(x)= -14.609
differential_evolution step 33: f(x)= -10.0074
differential_evolution step 33: f(x)= -12.0079
differential_evolution step 32: f(x)= -6.46436
differential_evolution step 3: f(x)= -5.16877
differential_evolution step 3: f(x)= -5.51382
differential_evolution step 4: f(x)= -13.0482
differential_evolution step 35: f(x)= -14.609
differential_evolution step 34: f(x)= -10.0074
differential_evolution step 34: f(x)= -12.0079
differential_evolution step 33: f(x)= -6.46436
differential_evolution step 4: f(x)= -5.30115
differential_evolution step 36: f(x)= -14.6976
differential_evolution step 35: f(x)= -10.0074
differential_evolution step 4: f(x)= -5.16491
differential_evolution step 35: f(x)= -12.0079
differential_evolution step 34: f(x)= -6.46436
differential_evolution step 4: f(x)= -3.70566
differential_evolution step 36: f(x)= -10.0074
differential_evolution step 4: f(x)= -6.24065
differential_evolution step 37: f(x)= -14.6976
differential_evolution

differential_evolution step 7: f(x)= -10.217
differential_evolution step 7: f(x)= -5.41966
differential_evolution step 61: f(x)= -12.9326
differential_evolution step 63: f(x)= -10.2823
differential_evolution step 64: f(x)= -16.1876
differential_evolution step 62: f(x)= -12.9326
differential_evolution step 64: f(x)= -10.2823
differential_evolution step 65: f(x)= -16.2567
differential_evolution step 63: f(x)= -12.9326
differential_evolution step 65: f(x)= -10.5928
differential_evolution step 66: f(x)= -16.5256
differential_evolution step 64: f(x)= -12.9326
differential_evolution step 7: f(x)= -4.16588
differential_evolution step 8: f(x)= -5.58667
differential_evolution step 67: f(x)= -16.5996
differential_evolution step 66: f(x)= -10.5928
differential_evolution step 8: f(x)= -3.70566
differential_evolution step 65: f(x)= -12.9326
differential_evolution step 8: f(x)= -17.042
differential_evolution step 68: f(x)= -16.7416
differential_evolution step 7: f(x)= -11.705
differential_evolution 

differential_evolution step 2: f(x)= -7.13408
differential_evolution step 13: f(x)= -5.67714
differential_evolution step 14: f(x)= -12.8612
differential_evolution step 6: f(x)= -5.10355
differential_evolution step 14: f(x)= -9.96597
differential_evolution step 2: f(x)= -8.06811
differential_evolution step 14: f(x)= -12.2819
differential_evolution step 14: f(x)= -17.042
differential_evolution step 13: f(x)= -6.15598
differential_evolution step 14: f(x)= -7.09946
differential_evolution step 3: f(x)= -5.6574
differential_evolution step 13: f(x)= -6.73041
differential_evolution step 14: f(x)= -4.52156
differential_evolution step 14: f(x)= -7.47938
differential_evolution step 13: f(x)= -5.8987
differential_evolution step 13: f(x)= -11.79
differential_evolution step 14: f(x)= -6.53679
differential_evolution step 14: f(x)= -9.94911
differential_evolution step 15: f(x)= -6.17457
differential_evolution step 3: f(x)= -7.15366
differential_evolution step 15: f(x)= -4.1869
differential_evolution s

differential_evolution step 22: f(x)= -7.09946
differential_evolution step 23: f(x)= -6.52545
differential_evolution step 20: f(x)= -6.19241
differential_evolution step 23: f(x)= -12.8612
differential_evolution step 22: f(x)= -8.96441
differential_evolution step 11: f(x)= -7.16797
differential_evolution step 10: f(x)= -12.3645
differential_evolution step 22: f(x)= -4.39893
differential_evolution step 22: f(x)= -7.71851
differential_evolution step 22: f(x)= -9.94911
differential_evolution step 23: f(x)= -10.5284
differential_evolution step 12: f(x)= -7.16044
differential_evolution step 22: f(x)= -6.23071
differential_evolution step 22: f(x)= -12.2819
differential_evolution step 23: f(x)= -20.0739
differential_evolution step 22: f(x)= -7.75422
differential_evolution step 22: f(x)= -4.52156
differential_evolution step 22: f(x)= -6.15598
differential_evolution step 15: f(x)= -6.34529
differential_evolution step 22: f(x)= -11.8876
differential_evolution step 22: f(x)= -7.28922
differential_

...

differential_evolution step 63: f(x)= -12.7983
differential_evolution step 64: f(x)= -11.2468
differential_evolution step 4: f(x)= -7.3868
differential_evolution step 58: f(x)= -6.40859
differential_evolution step 1: f(x)= -7.3259
differential_evolution step 59: f(x)= -6.40859
differential_evolution step 15: f(x)= -5.69131
differential_evolution step 2: f(x)= -9.17111
differential_evolution step 21: f(x)= -6.87285
differential_evolution step 63: f(x)= -5.90135
differential_evolution step 60: f(x)= -6.40859
differential_evolution step 3: f(x)= -9.17111
differential_evolution step 65: f(x)= -17.2729
differential_evolution step 61: f(x)= -6.40859
differential_evolution step 4: f(x)= -9.17111
differential_evolution step 64: f(x)= -9.39816
differential_evolution step 62: f(x)= -7.10786
differential_evolution step 17: f(x)= -9.06558
differential_evolution step 61: f(x)= -7.38158
differential_evolution step 54: f(x)= -8.41331
differential_evolution step 62: f(x)= -6.40859
differential_evoluti

differential_evolution step 9: f(x)= -8.44977
differential_evolution step 57: f(x)= -11.6824
differential_evolution step 10: f(x)= -7.87678
differential_evolution step 26: f(x)= -6.87285
differential_evolution step 9: f(x)= -12.1219
differential_evolution step 58: f(x)= -11.8646
differential_evolution step 59: f(x)= -11.8646
differential_evolution step 60: f(x)= -8.54529
differential_evolution step 60: f(x)= -11.8646
differential_evolution step 23: f(x)= -10.6372
differential_evolution step 61: f(x)= -11.8646
differential_evolution step 69: f(x)= -6.31986
differential_evolution step 2: f(x)= -5.70916
differential_evolution step 16: f(x)= -7.15984
differential_evolution step 62: f(x)= -12.037
differential_evolution step 70: f(x)= -11.4454
differential_evolution step 63: f(x)= -12.9405
differential_evolution step 68: f(x)= -7.10786
differential_evolution step 64: f(x)= -12.9405
differential_evolution step 21: f(x)= -6.01896
differential_evolution step 65: f(x)= -12.9405
differential_evol

differential_evolution step 67: f(x)= -8.67938
differential_evolution step 19: f(x)= -8.30501
differential_evolution step 11: f(x)= -11.7644
differential_evolution step 18: f(x)= -14.8113
differential_evolution step 18: f(x)= -11.7929
differential_evolution step 34: f(x)= -6.87285
differential_evolution step 69: f(x)= -8.56102
differential_evolution step 77: f(x)= -7.24574
differential_evolution step 11: f(x)= -8.18785
differential_evolution step 78: f(x)= -6.31986
differential_evolution step 25: f(x)= -7.15984
differential_evolution step 29: f(x)= -6.78134
differential_evolution step 79: f(x)= -14.0325
differential_evolution step 33: f(x)= -11.1387
differential_evolution step 20: f(x)= -8.31869
differential_evolution step 68: f(x)= -8.67938
differential_evolution step 12: f(x)= -11.7644
differential_evolution step 19: f(x)= -14.9032
differential_evolution step 19: f(x)= -11.7929
differential_evolution step 35: f(x)= -6.87285
differential_evolution step 70: f(x)= -8.56805
	Outputting l

	Outputting ligand to: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x478/autobuilt/initial_build_BAZ2BA-x478_5.0.pdb
differential_evolution step 39: f(x)= -7.15984
differential_evolution step 41: f(x)= -8.0261
differential_evolution step 33: f(x)= -16.999
differential_evolution step 47: f(x)= -7.22997
differential_evolution step 33: f(x)= -11.841
differential_evolution step 35: f(x)= -9.53496
differential_evolution step 40: f(x)= -7.15984
differential_evolution step 42: f(x)= -8.0261
differential_evolution step 26: f(x)= -8.7313
differential_evolution step 27: f(x)= -11.7644
differential_evolution step 48: f(x)= -7.22997
differential_evolution step 34: f(x)= -16.999
differential_evolution step 34: f(x)= -11.841
differential_evolution step 36: f(x)= -9.53496
differential_evolution step 27: f(x)= -8.7313
differential_evolution step 28: f(x)= -11.7644
differential_evolution step 41: f(x)= -7.15984
differential_evolution ste

differential_evolution step 54: f(x)= -21.2217
differential_evolution step 55: f(x)= -13.1206
	Outputting ligand to: /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x536/autobuilt/initial_build_BAZ2BA-x536_1.0.pdb
differential_evolution step 63: f(x)= -8.37944
differential_evolution step 71: f(x)= -7.22997
differential_evolution step 49: f(x)= -10.3194
differential_evolution step 50: f(x)= -12.0839
differential_evolution step 63: f(x)= -8.53093
differential_evolution step 56: f(x)= -13.1206
differential_evolution step 55: f(x)= -21.2217
differential_evolution step 64: f(x)= -8.37944
differential_evolution step 72: f(x)= -7.22997
differential_evolution step 50: f(x)= -10.3194
differential_evolution step 64: f(x)= -8.53093
differential_evolution step 57: f(x)= -13.1206
differential_evolution step 51: f(x)= -12.0839
differential_evolution step 56: f(x)= -21.2217
differential_evolution step 65: f(x)= -8.37944
differential_evolu

In [21]:
stripper = StripProtein()

stripped_protein_model_path = stripper(protein_model_path=str(tree["processed_datasets"][dtag]["initial_model"]()),
                               output_path=tree["processed_datasets"][dtag]["autobuilt"]["stripped_protein"](),
                              )

In [22]:
# from functools import partial

# autobuilders = {}
# for index, event in events_table_with_sites.iterrows():

#     print("index: {}".format(index))
#     print("event: {}".format(event))
#     dtag = index[0]
#     analysed_resolution = event["analysed_resolution"]
#     bdc = event["1-BDC"]
#     event_idx = int(float(index[1]))

#     ligand_model_paths = list(tree["processed_datasets"][dtag]["ligand_files"]().glob("*.pdb"))
    
#     if len(ligand_model_paths) >= 1:
#         ligand_model_path = str(ligand_model_paths[0])
#     else:
#         print("\tNO LIGANDS!")
#         continue

#     event_map_path=tree["processed_datasets"][dtag]["event_map"]([dtag,
#                                                                       event_idx,
#                                                                       bdc,
#                                                                       ],
#                                                                      )
#     output_dir_path = tree["processed_datasets"][dtag]["autobuilt"]["event"]([index[0], index[1]])

#     autobuilders[(dtag, int(event_idx))] = partial(autobuilder,
#                                       protein_model_path=stripped_protein_model_path,
#                                           ligand_model_path=placed_ligand_paths[index], 
#                                           event_map_path=event_map_path,
#                                           output_dir_path=output_dir_path,
#                                           resolution=analysed_resolution,
#                                           event=event,
#                                                    index=index,
#                                          )
    

In [60]:
refiners = {}
for index, event in events_table_with_sites.iterrows():
    
    if len(ligand_model_paths) >= 1:
        ligand_model_path = str(ligand_model_paths[0])
    else:
        print("\tNO LIGANDS!")
        continue

    dtag = index[0]
    analysed_resolution = event["analysed_resolution"]
    bdc = event["1-BDC"]
    event_idx = int(float(index[1]))

    ligand_model_paths = list(tree["processed_datasets"][dtag]["ligand_files"]().glob("*.pdb"))
    
    event_map_path=tree["processed_datasets"][dtag]["event_map"]([dtag,
                                                                      event_idx,
                                                                      bdc,
                                                                      ],
                                                                     )
    output_dir_path = tree["processed_datasets"][dtag]["autobuilt"]["event"]([index[0], index[1]])


    refiners[(dtag, int(event_idx))] = partial(RealSpaceRefinePhenix(),
                                               event_map_path=event_map_path,
                                               resolution=analysed_resolution,
                                               placed_ligand_model_path=placed_ligand_paths[index],
                                               output_dir_path=output_dir_path,
                                              )

In [61]:
# list(refiners.values())[0]()

Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/autobuilt/event_BAZ2BA-x463_4.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/autobuilt/initial_build_BAZ2BA-x463_4.0.pdb output_file_name=ready; phenix.real_space_refine /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/BAZ2BA-x463-event_4_1-BDC_0.24_map.ccp4 ready.pdb ready.cif resolution=1.79387787981


In [63]:
refinements = processer_building(refiners)

Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/autobuilt/event_BAZ2BA-x463_4.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/autobuilt/initial_build_BAZ2BA-x463_4.0.pdb output_file_name=ready; phenix.real_space_refine /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/BAZ2BA-x463-event_4_1-BDC_0.24_map.ccp4 ready.pdb ready.cif resolution=1.79387787981
Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x434/autobuilt/event_BAZ2BA-x434_1.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x434/autobuilt/initial_build_BAZ2BA-x434_1.0.pdb output_file_name=ready; phen

Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/autobuilt/event_BAZ2BA-x463_3.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/autobuilt/initial_build_BAZ2BA-x463_3.0.pdb output_file_name=ready; phenix.real_space_refine /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x463/BAZ2BA-x463-event_3_1-BDC_0.3_map.ccp4 ready.pdb ready.cif resolution=1.79387787981
Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x489/autobuilt/event_BAZ2BA-x489_1.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x489/autobuilt/initial_build_BAZ2BA-x489_1.0.pdb output_file_name=ready; pheni

Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x481/autobuilt/event_BAZ2BA-x481_1.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x481/autobuilt/initial_build_BAZ2BA-x481_1.0.pdb output_file_name=ready; phenix.real_space_refine /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x481/BAZ2BA-x481-event_1_1-BDC_0.32_map.ccp4 ready.pdb ready.cif resolution=1.7125639429
Run string: cd /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x478/autobuilt/event_BAZ2BA-x478_4.0; module load phenix; phenix.ready_set /dls/science/groups/i04-1/conor_dev/experiments/pandda_2_autobuild/BAZ2B/pandda_test/processed_datasets/BAZ2BA-x478/autobuilt/initial_build_BAZ2BA-x478_4.0.pdb output_file_name=ready; pheni